In [ ]:
import os

os.chdir('/Workspace/Shared/ag_vision')
%pip install .

In [ ]:
from pyspark.sql import SparkSession
from tqdm import tqdm

# Assumes you have a SparkSession named 'spark' available
spark = SparkSession.builder.getOrCreate()

In [ ]:
from ag_vision.constants import paths as pth
from ag_vision.images import field_images as fi
from ag_vision.pipelines.image_processing import add_metadata_to_field_image

## Read in the table generated in example 1.

In [ ]:
imgs_df = spark.table("use1_prod_artemis_catalog_3718194974443840.production.images_table").toPandas()

In [ ]:
len(imgs_df)

## Only process the ones that are missing metadata.

In [ ]:
# we only need to add metadata to images that don't have metadata.
imgs_df = imgs_df[imgs_df['metadata_path'].isna()]
print(len(imgs_df))

## Loop over all the images that are missing metadata, generate the files and generate the attitubes.

In [1]:
failed = []
for row in tqdm(imgs_df.itertuples(), total=len(imgs_df)):
    img_path = row[1]
    try:
        metadata_path = pth.generate_metadata_path_from_file_name(row.image_path)

        field_img = fi.FieldImage(cloud_img_key=row.image_path,
                                  cloud_metadata_key=metadata_path,
                                  platform='db',
                                  cloud_bucket='databricks')

        field_img.read_metadata()

        if field_img.metadata is None:
            field_img.initialize_metadata()

        field_img = add_metadata_to_field_image(field_img)

        field_img.save_metadata()

    except Exception as e:
        failed.append(row)
        print(f'Fail {e}')

NameError: name 'tqdm' is not defined